In [2]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import t, norm, chi2, chi2_contingency

from matplotlib import rc
rc('font', family='Malgun Gothic')      #한글 폰트설정
plt.rcParams['axes.unicode_minus']=False      #마이너스 부호 출력 설정

In [3]:
import math, scipy, sympy
from bs4 import BeautifulSoup as BS
import requests, re, urllib

from urllib import robotparser, request
from urllib.request import urlopen
import chardet

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

import os, sys, json

import cx_Oracle

In [4]:
from sklearn.datasets import load_iris
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV

### 사이킷런의 model_selection 모듈
model_selection 모듈은 머신러닝 모델을 개발하고 평가하는 데 유용한 도구입니다. model_selection 모듈을 사용하면 데이터를 더 잘 이해하고 더 나은 모델을 개발할 수 있습니다.
- 학습 데이터와 테스트 데이터를 분리하거나 교차 검증, 그리고 Estimator의 하이퍼 파라미터를 튜닝하기 위해서 다양한 함수와 클래스를 제공합니다.

- 학습 데이터와 테스트 데이터를 분리하는 방법에는 여러 가지가 있습니다. 가장 일반적인 방법은 train_test_split() 함수를 사용하는 것입니다. train_test_split() 함수는 학습 데이터와 테스트 데이터를 70:30의 비율로 분리합니다.

- 교차 검증은 모델의 성능을 평가하는 방법입니다. 교차 검증은 데이터를 여러 개의 폴드로 분리한 다음 각 폴드를 테스트 데이터로 사용하고 나머지 폴드를 학습 데이터로 사용하여 모델을 학습하는 방법입니다. 교차 검증은 모델의 성능을 더 정확하게 평가할 수 있습니다.

- Estimator의 하이퍼 파라미터를 튜닝하는 방법에는 여러 가지가 있습니다. 가장 일반적인 방법은 GridSearchCV() 함수를 사용하는 것입니다. GridSearchCV() 함수는 주어진 하이퍼 파라미터의 모든 조합에 대해 모델을 학습하고 평가하여 가장 좋은 하이퍼 파라미터를 찾습니다.

In [5]:
iris = load_iris()

train_data = iris.data
train_label = iris.target

dt_clf = DecisionTreeClassifier()   # 객체생성(분류기)
dt_clf.fit(train_data, train_label)   # 학습수행

pred = dt_clf.predict(train_data)   # 학습 데이터 셋으로 예측 수행
print('예측 정확도:', accuracy_score(train_label, pred))

예측 정확도: 1.0


In [6]:
# 학습 및 검증 데이터를 7:3으로 나누어 모델링 및 평가를 수행
iris_data = load_iris()

x_train, x_test, y_train, y_test, = train_test_split(
    iris_data.data, iris_data.target, test_size=0.3, random_state=66)
# train_test_split: 데이터셋, 데이터셋 라벨, 테스트 데이터셋 크기(30%), 시드값

dt_clf.fit(x_train, y_train)   # 학습수행
pred = dt_clf.predict(x_test)   # 학습 데이터 셋으로 예측 수행

print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9111


### 교차 검증이 필요한 이유

- 교차 유효성 검사는 독립적인 데이터 세트에서 모델의 성능을 평가하고 과적합을 방지하기 위해 기계 학습에 사용되는 통계 기법으로 다음과 같은 이유로 중요합니다.
    - 과적합: 과적합은 모델이 노이즈나 이상값을 포함하여 훈련 데이터를 너무 잘 학습할 때 발생합니다. 이러한 경우 모델은 교육 데이터에 대해 뛰어난 정확도를 갖지만 보이지 않는 데이터(학습하지 않은, 새로운 데이터)에 대해서는 성능이 저하될 수 있습니다. 교차 검증은 보다 일반화된 성능 메트릭을 제공하여 과적합을 방지하는 데 도움이 됩니다.
    - 하이퍼파라미터 조정: 교차 검증은 일반적으로 그리드 검색 또는 기타 하이퍼파라미터 튜닝 방법과 함께 사용되어 최상의 성능을 가진 모델을 생성하는 하이퍼파라미터를 찾습니다.
    - 모델 비교: 여러 모델을 비교할 때 교차 검증은 서로 다른 데이터 하위 집합의 성능을 평균화하므로 보다 신뢰할 수 있는 비교를 제공하는 데 도움이 됩니다.

- 교차 검증 방법 : k-겹 교차 검증
    - 데이터세트 분할: 전체 데이터세트를 'k'개의 동일한 부분(또는 '접기')으로 나눕니다. k=5를 선택한다고 가정해 보겠습니다. 즉, 데이터를 5개의 하위 집합으로 나눕니다.
    - 훈련 및 테스트: 하나의 하위 집합을 테스트 세트로 유지하고 나머지 k-1 하위 집합에서 모델을 훈련합니다.
    - 평가: 테스트 세트에서 모델의 성능 메트릭(예: 문제에 따라 정확도, 정밀도, 재현율, F1 점수 등)을 계산합니다.
    - 반복: 이 프로세스를 k번 반복합니다. 매번 다른 하위 집합을 테스트 세트로 사용하고 나머지 하위 집합을 훈련 세트로 사용합니다.
    - 평균 성능: k 반복에 대한 성능 메트릭의 평균을 계산합니다. 이는 모델의 교차 유효성 검사 점수로, 성능에 대한 보다 일반화된 척도를 제공합니다.

- 교차 유효성 검사의 기본 아이디어는 학습용 데이터에 대해 학습하고 검증용 데이터로 테스트할 때 얻은 "점수"가 새로운 데이터에서 모델이 수행되는 방식을 나타낼 가능성을 더 높다는 것입니다.
- Python에서는 sklearn.model_selection 모듈의 cross_val_score 함수를 사용하여 교차 검증을 쉽게 수행할 수 있습니다.

### 교차 검증 기술
- 기계 학습 모델을 학습할 때 학습 데이터에서 학습된 패턴을 기반으로 새로운 보이지 않는 데이터에 대해 정확한 예측을 할 수 있는 모델을 만들려고 합니다. 문제는 모델을 실제로 테스트하기 전에는 본 적이 없는 데이터에서 모델이 얼마나 잘 작동할지 알 수 없다는 것입니다.
- 교차 유효성 검사에는 전체 데이터 세트를 모델 훈련을 위한 더 큰 섹션과 모델 테스트를 위한 더 작은 섹션의 두 섹션으로 나누는 작업이 포함됩니다.
- '트레이닝 세트'는 데이터의 기본 패턴에 대해 모델을 가르치는 데 사용됩니다. 반면 '테스트 세트'는 모델이 이러한 패턴을 얼마나 잘 학습했는지 평가하는 데 사용됩니다. 모델은 교육 중에 테스트 데이터를 본 적이 없으므로 테스트 세트는 '보이지 않는 새로운 데이터'로 작동합니다.
- 이제 이 작은 부분(테스트 세트)에서 모델을 테스트할 때 얻은 "점수"는 모델이 미래에 실제 보이지 않는 데이터에서 얼마나 잘 수행될 것인지에 대한 추정치로 사용됩니다.
- 서로 다른 데이터 하위 집합(교차 유효성 검사의 핵심)에 대해 이 프로세스를 여러 번 반복함으로써 새 데이터로 일반화하는 모델의 능력에 대해 보다 신뢰할 수 있는 추정치를 얻습니다.

In [7]:
iris = load_iris()

features = iris.data
label = iris.target

dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드세트로 분리하는 KFold 객체와 
# 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)    #n_splits = k값
cv_acc = []

print('붓꽃 데이터 세트 크기:', features.shape[0])
features[:3]

붓꽃 데이터 세트 크기: 150


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [8]:
count=0
# KFold 객체의 split() 호출하면, 폴드별 학습용, 검증용 테스트의
# 로우 인덱스를 배열로 반환

for train_index, test_index in kfold.split(features):
    # KFold.split()으로 반환된 인덱스로 학습용, 검증용 테스트 데이터 추출
    x_train, x_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    count+=1
    
    # 반복시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print(f'''\n#{count}. 교차 검증 정확도: {accuracy}, 
    학습 데이터 크기: {train_size}, 
    검증 데이터 크기: {test_size}''')
    print(f'#{count}. 검증 세트 인덱스: {test_index}')
    
    cv_acc.append(accuracy)   #폴드 세트별 정확도를 담을 리스트

# 개별 iteration별 정확도를 합하여 평균 정확도 계산

print('\n### 평균 검증 정확도:', np.mean(cv_acc))


#1. 교차 검증 정확도: 1.0, 
    학습 데이터 크기: 120, 
    검증 데이터 크기: 30
#1. 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2. 교차 검증 정확도: 0.9667, 
    학습 데이터 크기: 120, 
    검증 데이터 크기: 30
#2. 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3. 교차 검증 정확도: 0.8667, 
    학습 데이터 크기: 120, 
    검증 데이터 크기: 30
#3. 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4. 교차 검증 정확도: 0.9333, 
    학습 데이터 크기: 120, 
    검증 데이터 크기: 30
#4. 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5. 교차 검증 정확도: 0.7333, 
    학습 데이터 크기: 120, 
    검증 데이터 크기: 30
#5. 검증 세트 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

### 평균 검증 정확도: 0.9


In [9]:
iris=load_iris()
iris_df=pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [10]:
# kfold 문제점: 한쪽에 치우치게 분포될 수 있음
# 학습 레이블은 모두 1,2이고, 검증 레이블은 모두 3으로 나뉠 수 있음
kfold = KFold(n_splits=3)

# kfild.split(x)는 폴드셋을 5번 반복할 때마다 달라지는 학습/테스트용 데이터 로우 인덱스 번호 반환
count=0

for train_idx, test_idx in kfold.split(iris_df):
    count+=1
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    print(f'##교차 검증_{count}')
    print(f'- 학습 레이블 데이터 분포: \n{label_train.value_counts()}')
    print(f'- 학습 레이블 데이터 분포: \n{label_test.value_counts()}\n')

##교차 검증_1
- 학습 레이블 데이터 분포: 
1    50
2    50
Name: label, dtype: int64
- 학습 레이블 데이터 분포: 
0    50
Name: label, dtype: int64

##교차 검증_2
- 학습 레이블 데이터 분포: 
0    50
2    50
Name: label, dtype: int64
- 학습 레이블 데이터 분포: 
1    50
Name: label, dtype: int64

##교차 검증_3
- 학습 레이블 데이터 분포: 
0    50
1    50
Name: label, dtype: int64
- 학습 레이블 데이터 분포: 
2    50
Name: label, dtype: int64



StratifiedKFold
- 클래스 불균형 문제가 있는 데이터 세트에 대해 교차 검증을 수행할 때 유용하게 사용될 수 있습니다. <br>(클래스 불균형: 한 클래스의 샘플 수가 다른 클래스의 샘플 수보다 훨씬 많은 경우)

- 이진 분류 문제를 생각해봅시다. 여기서 '양성' 클래스 샘플이 100개 있고, '음성' 클래스 샘플이 900개 있는 1000개의 샘플이 있다고 가정해보겠습니다. 이 경우, 데이터 세트는 '음성' 클래스 샘플이 대부분이므로 클래스 불균형이 있습니다.

- 일반적인 k-겹 교차 검증(KFold)를 사용하면, 일부 fold가 '양성' 클래스 샘플을 거의 포함하지 않을 수 있습니다. 이는 모델이 '양성' 클래스를 올바르게 학습하는 데 어려움을 줄 수 있습니다. 반면에 StratifiedKFold를 사용하면, 각 fold가 전체 데이터 세트의 클래스 분포를 대표하도록 만들 수 있습니다. 
- 즉, 각 fold는 '양성' 클래스 샘플의 약 10%와 '음성' 클래스 샘플의 약 90%를 포함하게 됩니다. 이렇게 하면 모델은 각 fold에서 '양성' 클래스와 '음성' 클래스를 모두 충분히 볼 수 있으므로, 클래스 불균형 문제를 완화할 수 있습니다.

- StratifiedKFold는 클래스 불균형 문제를 해결하는 데 도움이 될 수 있으며, 이는 모델의 성능을 개선하는 데 중요한 역할을 할 수 있습니다.

In [11]:
# StratifiedKFold는 원본 데이터의 레이블 분포를 고려해서 
# 이 분포와 동일하게 학습/검증 데이터 세트를 분배
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    
    print('## 교차 검증: {0}'.format(n_iter))
    print(f'학습 레이블 데이터 분포:\n{label_train.value_counts()}')
    print(f'검증 레이블 데이터 분포:\n{label_test.value_counts()}\n')

## 교차 검증: 1
학습 레이블 데이터 분포:
2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
0    17
1    17
2    16
Name: label, dtype: int64

## 교차 검증: 2
학습 레이블 데이터 분포:
1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
0    17
2    17
1    16
Name: label, dtype: int64

## 교차 검증: 3
학습 레이블 데이터 분포:
0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
1    17
2    17
0    16
Name: label, dtype: int64



In [12]:
from sklearn.model_selection import StratifiedKFold
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  

for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    x_train, x_test = features[train_index], features[test_index]   #학습 데이터, 평가 데이터
    y_train, y_test = label[train_index], label[test_index]   #출력 데이터(모델이 예측해야 하는 값), 정답 데이터(실제 값)
    
    dt_clf.fit(x_train , y_train)    #학습
    pred = dt_clf.predict(x_test)    #예측

    # 반복 시마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    print(f'''#[{n_iter}] 교차 검증 정확도: {accuracy}, 
     학습 데이터 크기: {train_size}, 
     검증 데이터 크기: {test_size}''')
    print(f'#[{n_iter}] 검증 세트 인덱스:{test_index}\n')
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy))

#[1] 교차 검증 정확도: 0.98, 
     학습 데이터 크기: 100, 
     검증 데이터 크기: 50
#[1] 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#[2] 교차 검증 정확도: 0.94, 
     학습 데이터 크기: 100, 
     검증 데이터 크기: 50
#[2] 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#[3] 교차 검증 정확도: 0.98, 
     학습 데이터 크기: 100, 
     검증 데이터 크기: 50
#[3] 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


#### cross_val_score( )
- Scikit-Learn의 cross_val_score() 함수는 기본적으로 회귀 모델에 대해서는 K-Fold 교차 검증 방법을, 분류 모델에 대해서는 Stratified K-Fold 교차 검증 방법을 사용합니다.
<br><br>
- 분류 모델에서 cross_val_score()를 사용하면, 데이터의 클래스 비율을 유지하며 데이터를 나누는 Stratified K-Fold 방식이 기본적으로 적용됩니다.
<br><br>
- 다만, 이는 기본 설정이며, 필요에 따라 cv 파라미터를 사용하여 다른 교차 검증 방법을 지정할 수 있습니다. 예를 들어, cv=KFold(n_splits=5) 와 같이 지정하면 5-Fold 교차 검증을 사용하게 됩니다.

In [13]:
from sklearn.model_selection import cross_val_score, cross_validate
# StratifiedKFold 정확도를 한 번에 뽑아낼 수 있음

iris = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris.data
label = iris.target

# 알고리즘은 디시전트리(dt_clf), 데이터(각 품종별 속성정보), 라벨(품종)
# 성능 지표는 정확도(scoring='accuracy'), 교차 검증 세트(cv)는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print(f'교차 검증별 정확도: {scores}')
print(f'평균 검증 정확도: {np.mean(scores)}')

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9666666666666667


### GridSearchCV
- GridSearchCV는 scikit-learn 라이브러리에서 제공하는 클래스로
- 지정된 하이퍼파라미터의 <<<모든 가능한 조합>>>에 대해 교차 검증을 수행하여 최적의 하이퍼파라미터를 찾는 방법입니다.
<br><br>
- 하이퍼파라미터는 머신러닝 모델의 성능을 조절하는 초매개변수들입니다. 예를 들어, 서포트 벡터 머신(SVM)에서는 마진의 넓이를 조절하는 "C"나 커널의 종류를 결정하는 "kernel" 등이 하이퍼파라미터가 됩니다. 이러한 하이퍼파라미터는 학습 데이터로부터 학습되는 것이 아니라, 사람이 직접 설정해주어야 합니다.
<br><br>
- GridSearchCV는 이러한 하이퍼파라미터를 효과적으로 설정하기 위한 도구입니다. 사용자가 지정한 범위 내에서 모든 하이퍼파라미터 조합을 시도하고, 각 조합에 대해 교차 검증을 수행하여 가장 성능이 좋은 조합을 찾습니다.
<br><br>
- GridSearchCV의 주요 하이퍼 파라미터는 다음과 같습니다:
    - estimator: 사용할 머신러닝 모델입니다.
    - param_grid: 하이퍼파라미터의 종류와 시도해 볼 값들을 사전 형태로 지정합니다.
    - scoring: 모델의 성능을 평가하는 지표를 지정합니다.
    - cv: 교차 검증을 수행할 때 데이터를 나누는 폴드의 개수를 지정합니다.
- GridSearchCV를 사용하면, 각각의 하이퍼파라미터 조합에 대해 성능을 자동으로 평가하므로, 수작업으로 각각의 조합을 시도해 볼 필요가 없습니다. 이는 시간을 절약하고, 또한 객체 지향 프로그래밍 방식을 사용하여 코드를 더욱 간결하고 이해하기 쉽게 만듭니다. 
- 하지만 GridSearchCV는 <<<모든 조합을 시도>>>하므로, 하이퍼파라미터의 개수나 시도해 볼 값의 개수가 많아지면 계산 비용이 크게 증가할 수 있습니다. 이러한 문제를 해결하기 위해 랜덤 서치(RandomizedSearchCV) 같은 다른 기법을 사용하기도 합니다.

In [14]:
from sklearn.model_selection import GridSearchCV
iris = load_iris()

# 학습데이터와 테스트 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(iris_data.data, 
                   iris_data.target, test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}
# min_samples_split: 분할을 시도하기 위해 노드(데이터셋을 분할하는 지점, 조건)가
# 가져야 하는 최소 샘플 수 (해당 노드의 데이터셋이 최소 n개 이상 되도록)
# 과적합 방지 목적

In [16]:
# param_grid의 하이퍼 파라미터들을 3개의 train, test 세트 폴드로 나눠서 테스트 수행 설정.  
# refit=True 가 디폴트(가장 좋은 파라미터 설정으로 재학습 시킴)
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가
grid_dtree.fit(x_train, y_train)

# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)

In [17]:
scores_df[['params', 'mean_test_score', 'rank_test_score',
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

# params: 하이퍼파라미터 조합
# mean_test_score: 평균 검증 점수
# rank_test_score: 하이퍼파라미터 조합의 성능 순위 (1이 가장 좋음)
# split(n)_test_score: 교차 검증 시 각 fold에서의 검증 점수

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [18]:
print(f'GridSearchCV 최적 하이퍼파라미터: {grid_dtree.best_params_}')
print(f'GridSearchCV 최고 정확도: {grid_dtree.best_score_:0.4f}')

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750


In [19]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼파라미터로 학습이 됨
pred = estimator.predict(x_test)
print(f'테스트 데이터 세트 정확도: {accuracy_score(y_test, pred):0.4f}')

테스트 데이터 세트 정확도: 0.9667


### 그리드 서치(Grid Search):

- 그리드 서치는 가능한 모든 하이퍼파라미터 조합을 지정된 탐색 공간에서 탐색합니다.
- 사용자가 사전에 지정한 하이퍼파라미터들의 모든 조합을 순차적으로 검증합니다.
- 모든 조합에 대해 교차 검증(Cross-Validation) 또는 검증 세트(Validation Set)를 사용하여 성능을 평가합니다.
- 모든 조합의 성능을 비교하여 최적의 하이퍼파라미터 조합을 찾습니다.
- 그리드 서치는 탐색 공간이 커질수록 가능한 조합이 급격히 증가하여 계산 비용이 크게 증가할 수 있습니다.
- 하이퍼파라미터 공간이 작고 가능한 조합이 제한적일 때 유용합니다.

### 랜덤 서치(Random Search):

- 랜덤 서치는 지정된 하이퍼파라미터 탐색 공간에서 무작위로 조합을 선택하여 탐색합니다.
- 사용자가 지정한 횟수 또는 시간 동안 조합을 무작위로 선택하여 검증합니다.
- 랜덤하게 선택되는 조합은 균일하지 않을 수 있지만, 다양한 조합을 탐색할 수 있습니다.
- 교차 검증 또는 검증 세트를 사용하여 성능을 평가합니다.
- 랜덤 서치는 탐색 공간이 크고 가능한 조합이 많을 때 유용합니다.
- 계산 비용을 조정하여 탐색 범위와 횟수를 조정할 수 있습니다.

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# 탐색할 하이퍼파라미터 범위 설정
param = {
    'n_estimators': randint(0, 100),  # 100에서 1000 사이의 값 중 랜덤 선택
    'max_depth': randint(1, 20),         # 5에서 20 사이의 값 중 랜덤 선택
    'min_samples_split': randint(2, 10),  # 2에서 10 사이의 값 중 랜덤 선택
    'min_samples_leaf': randint(1, 5)     # 1에서 5 사이의 값 중 랜덤 선택
}

# 랜덤 포레스트 분류기 객체 생성
rf = RandomForestClassifier()

# 랜덤 서치 수행
rs = RandomizedSearchCV(rf, param_distributions=param, 
                n_iter=10, cv=3)  #n_iter: n개 조합 무작위로 선택
rs.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 최고 성능 출력
print("RandomForest 최적 하이퍼파라미터:", rs.best_params_)
print(f"RandomForest 최고 예측 정확도: {rs.best_score_:.4f}")

RandomForest 최적 하이퍼파라미터: {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 36}
RandomForest 최고 예측 정확도: 0.9667


#### 과제(1): k값을 3,10으로 설정하여 각각 교차 검증을 수행하고, 검증 정확도가 가장 높은 k값을 선택하세요

- k=3일때, 평균 검증 정확도: 0.9533333333333333
- k=10일때, 평균 검증 정확도: 0.9599799999999998 (win!)

In [34]:
iris = load_iris()
feature = iris.data
label = iris.target

dtc = DecisionTreeClassifier(random_state=123)

kfold = KFold(n_splits=3, shuffle=True, random_state=123)
li_acc=[]

ct=0
for train_idx, test_idx in kfold.split(feature):
    x_train, x_test = feature[train_idx], feature[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    dtc.fit(x_train, y_train)   #fit: 학습
    pred = dtc.predict(x_test)   #predict: 예측
    ct+=1
    
    accuracy = accuracy_score(y_test, pred)
    accuracy = np.round(accuracy, 4)
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print(f'''{ct}. 교차 검증 정확도: {accuracy}
    학습 데이터 크기: {train_size}
    검증 데이터 크기: {test_size}''')
    print(f'{ct}. 검증 세트 인덱스: {test_idx}\n')
    
    li_acc.append(accuracy)
    
print(f"평균 검증 정확도: {np.mean(li_acc)}")

1. 교차 검증 정확도: 0.96
    학습 데이터 크기: 100
    검증 데이터 크기: 50
1. 검증 세트 인덱스: [  0   4   5   8  13  16  19  20  24  28  29  31  33  35  36  37  38  42
  45  46  53  59  60  62  63  72  82  87  88  90  93 104 112 114 116 117
 119 121 127 130 131 132 133 135 136 138 141 142 143 145]

2. 교차 검증 정확도: 0.94
    학습 데이터 크기: 100
    검증 데이터 크기: 50
2. 검증 세트 인덱스: [  1   6   9  10  11  12  14  15  18  21  22  23  25  26  30  41  43  44
  50  51  54  56  58  65  70  71  74  77  79  80  81  86  89  91  95 100
 101 103 107 108 110 115 118 122 124 128 129 139 146 149]

3. 교차 검증 정확도: 0.96
    학습 데이터 크기: 100
    검증 데이터 크기: 50
3. 검증 세트 인덱스: [  2   3   7  17  27  32  34  39  40  47  48  49  52  55  57  61  64  66
  67  68  69  73  75  76  78  83  84  85  92  94  96  97  98  99 102 105
 106 109 111 113 120 123 125 126 134 137 140 144 147 148]

평균 검증 정확도: 0.9533333333333333


In [35]:
iris = load_iris()
feature = iris.data
label = iris.target

dtc = DecisionTreeClassifier(random_state=123)

kfold = KFold(n_splits=10, shuffle=True, random_state=123)
li_acc=[]

ct=0
for train_idx, test_idx in kfold.split(feature):
    x_train, x_test = feature[train_idx], feature[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    dtc.fit(x_train, y_train)
    pred = dtc.predict(x_test)
    ct+=1
    
    accuracy = accuracy_score(y_test, pred)
    accuracy = np.round(accuracy, 4)
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print(f'''{ct}. 교차 검증 정확도: {accuracy}
    학습 데이터 크기: {train_size}
    검증 데이터 크기: {test_size}''')
    print(f'{ct}. 검증 세트 인덱스: {test_idx}\n')
    
    li_acc.append(accuracy)
    
print(f"평균 검증 정확도: {np.mean(li_acc)}")

1. 교차 검증 정확도: 0.9333
    학습 데이터 크기: 135
    검증 데이터 크기: 15
1. 검증 세트 인덱스: [  8  33  37  42  59  72  87  88  90 112 116 132 135 138 141]

2. 교차 검증 정확도: 1.0
    학습 데이터 크기: 135
    검증 데이터 크기: 15
2. 검증 세트 인덱스: [  4  13  20  24  28  31  36  45  46  63 104 117 121 127 133]

3. 교차 검증 정확도: 0.9333
    학습 데이터 크기: 135
    검증 데이터 크기: 15
3. 검증 세트 인덱스: [  0   5  16  29  35  60  62  82  93 119 130 131 142 143 145]

4. 교차 검증 정확도: 1.0
    학습 데이터 크기: 135
    검증 데이터 크기: 15
4. 검증 세트 인덱스: [  9  19  23  38  53  65  79  86  89  91 101 110 114 115 136]

5. 교차 검증 정확도: 0.9333
    학습 데이터 크기: 135
    검증 데이터 크기: 15
5. 검증 세트 인덱스: [ 11  12  21  41  44  51  74  81  95 103 118 122 124 129 149]

6. 교차 검증 정확도: 0.9333
    학습 데이터 크기: 135
    검증 데이터 크기: 15
6. 검증 세트 인덱스: [  1   6  14  25  26  43  50  54  56  70  71  80 100 128 146]

7. 교차 검증 정확도: 0.9333
    학습 데이터 크기: 135
    검증 데이터 크기: 15
7. 검증 세트 인덱스: [  3  10  15  18  22  30  40  58  64  69  75  77 107 108 139]

8. 교차 검증 정확도: 1.0
    학습 데이터 크기: 135
    검증 데이터 크기: 15
8. 검증 

#### 과제(1): 와인 데이터에 대해서 아래 사항을 고려하여 모델 생성 및 성능개선을 위한 하이퍼파라미터 튜닝을 수행하세요. (GridSearchCV)
- dt를 알고리즘으로 적용
- cv = 5
- param_grid = {'max_depth':[3,4,5,6],'min_samples_split':[2,3,4]}

In [2]:
# 와인 데이터 불러오기
from sklearn import datasets

wine = datasets.load_wine()
x, y = wine.data, wine.target

wine.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

In [3]:
print(x)
y

[[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

#### 데이터를 나누었을 경우
모델이 학습하지 않은 데이터에서의 성능 평가도 가능 = 모델의 일반화 성능 평가하기 적합

In [8]:
# 데이터셋 나누기
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                       test_size=0.3, random_state=121)

# 의사결정나무 모델 객체 생성
dt_clf = DecisionTreeClassifier(random_state=42)

# 하이퍼파라미터 후보군 정의
param_grid = {'max_depth': [3, 4, 5, 6], 'min_samples_split': [2, 3, 4]}

# 그리드서치 객체 생성
grid_dt = GridSearchCV(dt_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# scoring='accuracy', refit=True 기본값
# n_jobs=-1 사용 가능한 컴퓨터 코어 전부 사용

# 그리드서치 수행 (학습)
grid_dt.fit(x_train, y_train)

# 결과 출력
print('GridSearchCV 최적 파라미터:', grid_dt.best_params_)
print('GridSearchCV 최고 정확도:', grid_dt.best_score_)

scores_df = pd.DataFrame(grid_dt.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score',
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

GridSearchCV 최적 파라미터: {'max_depth': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9106666666666665


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 3, 'min_samples_split': 2}",0.902333,7,0.96,0.92,0.92
1,"{'max_depth': 3, 'min_samples_split': 3}",0.902333,7,0.96,0.92,0.92
2,"{'max_depth': 3, 'min_samples_split': 4}",0.902333,7,0.96,0.92,0.92
3,"{'max_depth': 4, 'min_samples_split': 2}",0.902333,7,0.96,0.92,0.92
4,"{'max_depth': 4, 'min_samples_split': 3}",0.902333,7,0.96,0.92,0.92
5,"{'max_depth': 4, 'min_samples_split': 4}",0.902333,7,0.96,0.92,0.92
6,"{'max_depth': 5, 'min_samples_split': 2}",0.910667,1,0.96,0.92,0.92
7,"{'max_depth': 5, 'min_samples_split': 3}",0.910667,1,0.96,0.92,0.92
8,"{'max_depth': 5, 'min_samples_split': 4}",0.910667,1,0.96,0.92,0.92
9,"{'max_depth': 6, 'min_samples_split': 2}",0.910667,1,0.96,0.92,0.92


In [9]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator반환
estimator = grid_dt.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼파라미터로 학습이 됨
pred = estimator.predict(x_test)
print(f'테스트 데이터 세트 정확도: {accuracy_score(y_test, pred):0.4f}')

테스트 데이터 세트 정확도: 0.8889


#### 데이터를 나누지 않았을 경우
모델의 일반화 성능을 평가하기 부적합

In [10]:
# 의사결정나무 모델 객체 생성
dt_clf = DecisionTreeClassifier(random_state=42)

# 하이퍼파라미터 후보군 정의
param_grid = {'max_depth': [3, 4, 5, 6], 'min_samples_split': [2, 3, 4]}

# 그리드서치 객체 생성
grid_dt = GridSearchCV(dt_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# n_jobs: 사용할 CPU 코어 수를 지정하는 매개변수 (-1이면 가능한 모든 CPU코어 사용)

# 모델 학습
grid_dt.fit(x, y)

# 결과 출력
print('GridSearchCV 최적 파라미터:', grid_dt.best_params_)
print('GridSearchCV 최고 정확도:', grid_dt.best_score_)

GridSearchCV 최적 파라미터: {'max_depth': 4, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.8936507936507937
